In [1]:
import requests
import pandas as pd
import time

In [ ]:
base_url = "https://uatapi.fwc.gov.au/api/v1/awards"

In [37]:
def get_all_results(query_url,hdr,payload):
        response = requests.get(url=query_url,headers=hdr,params=payload).json()
        page_count = response['_meta']['page_count']
        result_list=[]
        result_list.append(pd.DataFrame.from_dict(response['results']))

        # if # of page >=2, get all pages results
        if(page_count>1):
            for idx in range(2,page_count+1):
                page_paras = {
                    'page' : idx,
                    'limit': 100
                        }
                page_response = requests.get(url=query_url,headers=hdr,params=page_paras).json()
                result_list.append(pd.DataFrame.from_dict(page_response['results']))
                time.sleep(1)
        result_df = pd.concat(result_list)
        return result_df

In [50]:
class FWCAPI():

    def __init__(self,subscription_key):
        self.key = subscription_key
        self.hdr = {
        'Cache-Control': 'no-cache',
        'Ocp-Apim-Subscription-Key': self.key,
        }

    def get_awards(self,keywords,award_operative_from=None,award_operative_to=None):
        # https://uatdeveloper.fwc.gov.au/api-details#api=fwc-maapi-v1&operation=GetAwards
        # construct url
        query_url = base_url + "?"

        #construct payload to API
        payload = {}
        payload['name'] = keywords
        payload['award_operative_from']=award_operative_from
        payload['award_operative_to']=award_operative_to
        payload['limit']=100

        return get_all_results(query_url,self.hdr,payload)

    def get_award(self,award_code,award_operative_from=None,award_operative_to=None):
        # https://uatdeveloper.fwc.gov.au/api-details#api=fwc-maapi-v1&operation=AwardsIdentification
        # construct url
        query_url = base_url + "/" + award_code

        #construct payload to API
        payload = {}
        payload['award_operative_from']=award_operative_from
        payload['award_operative_to']=award_operative_to
        payload['limit']=100

        return get_all_results(query_url,self.hdr,payload)        

    def get_classification(self,award_code,classification_fixed_id=None):
        # https://uatdeveloper.fwc.gov.au/api-details#api=fwc-maapi-v1&operation=IndividualClassification
        # construct url
        query_url = f"{base_url}/{award_code}/classifications/{classification_fixed_id}"

        #construct payload to API
        payload = {}

        payload['classification_fixed_id']=classification_fixed_id
        payload['limit']=100

        return get_all_results(query_url,self.hdr,payload)   

    def get_payrates(self,award_code,classification_level=None,classification_fixed_id=None,employee_rate_type_code=None,operative_from=None,operative_to=None):
        # https://uatdeveloper.fwc.gov.au/api-details#api=fwc-maapi-v1&operation=GetPayRates
        # construct url
        query_url = f"{base_url}/{award_code}/pay-rates?"

        #construct payload to API
        payload = {}
        payload['classification_level']=classification_level
        payload['classification_fixed_id']=classification_fixed_id
        payload['employee_rate_type_code']=employee_rate_type_code
        payload['operative_from']=operative_from
        payload['operative_to']=operative_to
        payload['limit']=100

        return get_all_results(query_url,self.hdr,payload)       

    def get_classifications(self,award_code,classification_level=None,classification_fixed_id=None,operative_from=None,operative_to=None):
        # https://uatdeveloper.fwc.gov.au/api-details#api=fwc-maapi-v1&operation=AwardClassification
        # construct url
        query_url = f"{base_url}/{award_code}/classifications?"

        #construct payload to API
        payload = {}
        payload['classification_level']=classification_level
        payload['classification_fixed_id']=classification_fixed_id
        payload['operative_from']=operative_from
        payload['operative_to']=operative_to        
        payload['limit']=100

        return get_all_results(query_url,self.hdr,payload)    

    def get_current_payrate(self,award_code,classification_fixed_id=None):
        # https://uatdeveloper.fwc.gov.au/api-details#api=fwc-maapi-v1&operation=IndividualPayRate
        # construct url
        query_url = f"{base_url}/{award_code}/classifications/{classification_fixed_id}/pay-rates?"

        #construct payload to API
        payload = {}
        payload['classification_fixed_id']=classification_fixed_id   
        payload['limit']=100

        return get_all_results(query_url,self.hdr,payload)            



In [51]:
fwc = FWCAPI('5d37f1a91012490089e4a5ac698fd512')

In [ ]:
fwc.get_classifications('MA000001')

In [ ]:
fwc.get_payrates('MA000001')

In [44]:
# fwc.get_list_of_awards('mining','2000-01-01','2012-01-01')
fwc.get_list_of_awards('Black Coal Mining')

,award_id,award_fixed_id,code,name,award_operative_from,award_operative_to,version_number,last_modified_datetime,published_year
0,1132,1,MA000001,Black Coal Mining Industry Award 2020,2010-01-01T00:00:00+00:00,None,3,2022-01-17T16:12:10+00:00,2021


In [41]:
fwc.get_award('MA000001')

,award_id,award_fixed_id,code,name,award_operative_from,award_operative_to,version_number,last_modified_datetime,published_year
0,0,1,MA000001,Black Coal Mining Industry Award 2020,2010-01-01T00:00:00+00:00,None,3,2022-01-17T16:12:10+00:00,2021
1,0,1,MA000001,Black Coal Mining Industry Award 2020,2010-01-01T00:00:00+00:00,None,3,2021-06-26T11:33:16+00:00,2020
2,0,1,MA000001,Black Coal Mining Industry Award 2020,2010-01-01T00:00:00+00:00,None,3,2019-10-23T11:12:07+00:00,2019
3,0,1,MA000001,Black Coal Mining Industry Award 2020,2010-01-01T00:00:00+00:00,None,3,2019-10-11T13:06:14+00:00,2018
4,0,1,MA000001,Black Coal Mining Industry Award 2020,2010-01-01T00:00:00+00:00,None,3,2019-10-10T11:13:00+00:00,2017
5,0,1,MA000001,Black Coal Mining Industry Award 2020,2010-01-01T00:00:00+00:00,None,3,2019-10-04T17:54:28+00:00,2016
6,0,1,MA000001,Black Coal Mining Industry Award 2020,2010-01-01T00:00:00+00:00,None,3,2019-10-01T14:58:42+00:00,2015
